In [2]:
import pandas as pd


spot = pd.read_csv('../Idea_data/spot_cal.csv')
future = pd.read_csv('../Idea_data/future.cal.csv')
funding = pd.read_csv('../Idea_data/funding_daily.csv')



#data cleaning
spot.rename(columns={"open_time": "date"}, inplace=True)
future.rename(columns={"open_time": "date"}, inplace=True)

funding["date"] = pd.to_datetime(funding["date"]).dt.tz_localize(None)  # remove timezone if present
spot["date"] = pd.to_datetime(spot["date"])  # normal conversion
future["date"] = pd.to_datetime(future["date"])  # normal conversion


spot = spot.rename(columns={
    "volume": "spot_volume",
    "normal_return": "spot_return",
    "log_return": "spot_log_return",
    "30d_zscore_return": "spot_z30d_return",
    "30d_zscore_logreturn": "spot_z30d_logreturn",
    "30d_zscore": "spot_z30d_price",
    "30d_zscore_volume": "spot_z30d_volume",
    "30d_zscore_volume_change": "spot_z30d_vol_change",
    "7d_zscore": "spot_z7d_price",
    "7d_zscore_volume_change": "spot_z7d_vol_change"
})[[
    "date", "open", "high", "low", "close", "spot_volume",
    "spot_return", "spot_log_return", "spot_z30d_return",
    "spot_z30d_logreturn", "intraday", "z_intraday_30",
    "spot_z30d_price", "spot_z30d_volume", "spot_z30d_vol_change",
    "spot_z7d_price", "spot_z7d_vol_change", "ema200", "above_ema200"
]]


master = (
    spot
    .merge(future, on="date", how="inner")
    .merge(funding, on="date", how="inner")
)



master.columns

Index(['date', 'open_x', 'high_x', 'low_x', 'close_x', 'spot_volume',
       'spot_return', 'spot_log_return', 'spot_z30d_return',
       'spot_z30d_logreturn', 'intraday_x', 'z_intraday_30_x',
       'spot_z30d_price', 'spot_z30d_volume', 'spot_z30d_vol_change',
       'spot_z7d_price', 'spot_z7d_vol_change', 'ema200', 'above_ema200',
       'Unnamed: 0_x', 'open_y', 'high_y', 'low_y', 'close_y', 'volume',
       'close_time', 'quote_volume', 'count', 'taker_buy_volume',
       'taker_buy_quote_volume', 'prep_normal_return', 'Z_prep_30d_return',
       'prep_log_return', 'Z_prep30d_logreturn', 'prep_7d_rolling_mean',
       'prep_7d_rolling_volatility', 'z_prep_7d', 'prep_30d_rolling_mean',
       'prep_30d_rolling_volatility', 'z_prep_30d', 'intraday_y',
       'z_intraday_30_y', 'z_prep_volume', 'prep_volume_change_%',
       'z_prep_30d_volume_change', 'z_prep_7d_volume_change',
       'taker_sell_volume', 'buy_share', 'sell_share', 'OFI', 'z_prep_OFI',
       'prep_average_trade_v

In [5]:

# Drop duplicates and irrelevant columns
drop_cols = [
    "open_x", "high_x", "low_x", "close_x",   # drop spot OHLC, keep futures OHLC
    "intraday_x", "intraday_y",               # consolidate later if needed
    "z_intraday_30_x", "z_intraday_30_y",     # redundant, very high correlation
    "Unnamed: 0_x", "Unnamed: 0_y",           # index junk
    "count", "close_time"                     # metadata
]

master = master.drop(columns=drop_cols, errors="ignore")

# Rename for clarity
master = master.rename(columns={
    "open_y": "open",
    "high_y": "high",
    "low_y": "low",
    "close_y": "close",
    "volume": "prep_volume"   # differentiate futures vs spot volume
})

print(master.columns)

master.to_csv('../Idea_data/master_data.csv', index=False)



Index(['date', 'spot_volume', 'spot_return', 'spot_log_return',
       'spot_z30d_return', 'spot_z30d_logreturn', 'spot_z30d_price',
       'spot_z30d_volume', 'spot_z30d_vol_change', 'spot_z7d_price',
       'spot_z7d_vol_change', 'ema200', 'above_ema200', 'open', 'high', 'low',
       'close', 'prep_volume', 'quote_volume', 'taker_buy_volume',
       'taker_buy_quote_volume', 'prep_normal_return', 'Z_prep_30d_return',
       'prep_log_return', 'Z_prep30d_logreturn', 'prep_7d_rolling_mean',
       'prep_7d_rolling_volatility', 'z_prep_7d', 'prep_30d_rolling_mean',
       'prep_30d_rolling_volatility', 'z_prep_30d', 'z_prep_volume',
       'prep_volume_change_%', 'z_prep_30d_volume_change',
       'z_prep_7d_volume_change', 'taker_sell_volume', 'buy_share',
       'sell_share', 'OFI', 'z_prep_OFI', 'prep_average_trade_volume_inU',
       'prep_average_trade_volume_inbtc', 'z_prep_30d_average_trade_inU',
       'funding_mean', 'funding_first', 'funding_last', 'funding_std',
       'z30_

In [12]:
print(master.iloc[51])


date                               2025-05-27 00:00:00
spot_volume                                21276.65635
spot_return                                  -0.453804
spot_log_return                              -0.004548
spot_z30d_return                                   NaN
spot_z30d_logreturn                                NaN
spot_z30d_price                                    NaN
spot_z30d_volume                                   NaN
spot_z30d_vol_change                               NaN
spot_z7d_price                                -0.12353
spot_z7d_vol_change                           0.803361
ema200                                    98504.512038
above_ema200                                      True
open                                          109386.8
high                                          110700.0
low                                           107438.0
close                                         108889.3
prep_volume                                 223599.905
quote_volu